In [1]:
import os
import torch
import yaml
from addict import Dict
from model_lightning_seg import MyModel

# === Configura tus rutas ===
ckpt_dir = "/data/GitHub/Breast-AI-model/experiment_5_giancarlo"
config_path = "/data/GitHub/Breast-AI-model/src/default_config_train_seg.yaml"
onnx_out_dir = "./onnx_models"
os.makedirs(onnx_out_dir, exist_ok=True)

# === Carga el config YAML ===
with open(config_path, "r") as f:
    conf = Dict(yaml.safe_load(f))

# === Encuentra los .ckpt ===
ckpts = sorted([
    os.path.join(ckpt_dir, f)
    for f in os.listdir(ckpt_dir)
    if f.endswith(".ckpt")
])

if not ckpts:
    raise FileNotFoundError(f"No .ckpt found in {ckpt_dir}")

# === Exporta a ONNX con Lightning ===
for i, ckpt_path in enumerate(ckpts):
    print(f"📦 Exportando modelo {i+1}/{len(ckpts)}: {ckpt_path}")
    
    model = MyModel.load_from_checkpoint(
        checkpoint_path=ckpt_path,
        model_opts=conf.model_opts,
        train_par=conf.train_par,
        strict=False,
    )
    model.eval()

    # Dummy input: (1, 1, 128, 128, 128)
    dummy_input = torch.randn(1, 1, 128, 128, 128, dtype=torch.float32)

    # Ruta de salida ONNX
    onnx_path = os.path.join(onnx_out_dir, f"model_fold{i}.onnx")

    # Exportar usando Lightning
    model.to_onnx(
        file_path=onnx_path,
        input_sample=dummy_input,
        export_params=True,
        opset_version=17,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    )

    print(f"✅ Guardado: {onnx_path}\n")

print("🎉 Conversión a ONNX completada.")


📦 Exportando modelo 1/10: /data/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_1.ckpt


/data/GitHub/Breast-AI-model/src/models_arq/att_unet_seg.py:45: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x_up.shape[2:] != y.shape[2:]:
/home/exx/anaconda3/envs/ligthning/lib/python3.12/site-packages/torch/onnx/symbolic_helper.py:1460: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'instance_norm' is set to train=True. Exporting with train=True.
  warnings.warn(


✅ Guardado: ./onnx_models/model_fold0.onnx

📦 Exportando modelo 2/10: /data/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_10.ckpt
✅ Guardado: ./onnx_models/model_fold1.onnx

📦 Exportando modelo 3/10: /data/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_2.ckpt
✅ Guardado: ./onnx_models/model_fold2.onnx

📦 Exportando modelo 4/10: /data/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_3.ckpt
✅ Guardado: ./onnx_models/model_fold3.onnx

📦 Exportando modelo 5/10: /data/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_4.ckpt
✅ Guardado: ./onnx_models/model_fold4.onnx

📦 Exportando modelo 6/10: /data/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_5.ckpt
✅ Guardado: ./onnx_models/model_fold5.onnx

📦 Exportando modelo 7/10: /data/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_6.ckpt
✅ Guardado: ./onnx_models/model_fold6.onnx

📦 Exportando modelo 8/10: /data/GitHub/Breast-AI-model/experi

In [11]:
import os
import torch
from model_lightning_clasi import MyModelMulticlass
from addict import Dict

# === Rutas ===
ckpt_dir = "D:/Research Projects/Breast-AI-model/Classification_ckpts"
onnx_out_dir = "D:/Research Projects/Breast-AI-model/Classification_onnx"
os.makedirs(onnx_out_dir, exist_ok=True)

# === Lista de nombres modelo conocidos (ajustar si agregas más)
model_names = ['densenet', 'mobilenet', 'simple', 'vgg16']

# === Parámetros de entrenamiento
train_par = Dict({'eval_threshold': 0.5, 'loss_opts': {'name': 'CrossEntropyLoss'}})

# === Buscar checkpoints
ckpts = sorted([
    os.path.join(ckpt_dir, f)
    for f in os.listdir(ckpt_dir)
    if f.endswith(".ckpt")
])
if not ckpts:
    raise FileNotFoundError(f"No .ckpt found in {ckpt_dir}")

# === Exportación robusta
def export_to_onnx(model, dummy_input, output_path):
    model.eval()
    with torch.no_grad():
        torch.onnx.export(
            model,
            dummy_input,
            output_path,
            export_params=True,
            opset_version=17,
            input_names=["input"],
            output_names=["output"],
            dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
            do_constant_folding=True
        )

# === Procesar cada modelo
for i, ckpt_path in enumerate(ckpts):
    model_name = None
    for name in model_names:
        if name in ckpt_path.lower():
            model_name = name
            break
    if model_name is None:
        print(f"⚠️ No se pudo determinar el tipo de modelo para {ckpt_path}. Saltando.")
        continue

    print(f"📦 Exportando modelo {i+1}/{len(ckpts)}: {model_name} - {os.path.basename(ckpt_path)}")

    # Cargar modelo
    model_opts = Dict({'name': model_name})
    model = MyModelMulticlass.load_from_checkpoint(
        checkpoint_path=ckpt_path,
        model_opts=model_opts,
        train_par=train_par,
    )
    model.eval()

    # Detectar dispositivo y mover modelo e input
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    dummy_input = torch.randn(1, 3, 224, 224).to(device)

    # Ruta de salida
    onnx_path = os.path.join(onnx_out_dir, f"classifier_{i}_{model_name}.onnx")

    try:
        export_to_onnx(model, dummy_input, onnx_path)
        print(f"✅ Guardado: {onnx_path}\n")
    except Exception as e:
        print(f"❌ Error al exportar {onnx_path}: {e}")

print("🎉 Conversión a ONNX finalizada.")



📦 Exportando modelo 1/3: densenet - densenet.ckpt
✅ Guardado: D:/Research Projects/Breast-AI-model/Classification_onnx\classifier_0_densenet.onnx

📦 Exportando modelo 2/3: mobilenet - mobilenet.ckpt
✅ Guardado: D:/Research Projects/Breast-AI-model/Classification_onnx\classifier_1_mobilenet.onnx

📦 Exportando modelo 3/3: vgg16 - vgg16.ckpt


c:\Users\Luca\miniconda3\envs\testenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ Guardado: D:/Research Projects/Breast-AI-model/Classification_onnx\classifier_2_vgg16.onnx

🎉 Conversión a ONNX finalizada.


In [12]:
import onnx

# Ruta al modelo ONNX
onnx_path = "D:/Research Projects/Breast-AI-model/Classification_onnx/classifier_0_densenet.onnx"

# Cargar y verificar
model = onnx.load(onnx_path)
onnx.checker.check_model(model)

print("✅ Modelo ONNX válido.")


✅ Modelo ONNX válido.


In [13]:
import onnxruntime as ort
import numpy as np

# Ruta del modelo
onnx_path = "D:/Research Projects/Breast-AI-model/Classification_onnx/classifier_0_densenet.onnx"

# Crear sesión
ort_session = ort.InferenceSession(onnx_path)

# Dummy input (igual que el usado al exportar)
dummy_input = np.random.randn(1, 3, 224, 224).astype(np.float32)

# Ejecutar inferencia
outputs = ort_session.run(None, {"input": dummy_input})

print("✅ Inferencia OK. Salida:")
print(outputs[0])


✅ Inferencia OK. Salida:
[[-1.5368236   2.1265516  -0.38121608]]


In [15]:
import torch
import numpy as np
import onnx
import onnxruntime as ort

from model_lightning_clasi import MyModelMulticlass
from addict import Dict

# === Rutas ===
ckpt_path = "D:/Research Projects/Breast-AI-model/Classification_ckpts/densenet.ckpt"
onnx_path = "D:/Research Projects/Breast-AI-model/Classification_onnx/classifier_0_densenet.onnx"

# === Configuración ===
model_opts = Dict({'name': 'densenet'})
train_par = Dict({'eval_threshold': 0.5, 'loss_opts': {'name': 'CrossEntropyLoss'}})

# === Dummy input ===
dummy_input_torch = torch.randn(1, 3, 224, 224)

# === PyTorch inference ===
model = MyModelMulticlass.load_from_checkpoint(
    checkpoint_path=ckpt_path,
    model_opts=model_opts,
    train_par=train_par,
)
model = model.cpu()
model.eval()

with torch.no_grad():
    torch_output = model(dummy_input_torch)
    torch_output_np = torch_output.cpu().numpy()

# === ONNX inference ===
ort_session = ort.InferenceSession(onnx_path)
dummy_input_numpy = dummy_input_torch.numpy().astype(np.float32)

onnx_output = ort_session.run(None, {"input": dummy_input_numpy})[0]

# === Comparación ===
np.testing.assert_allclose(torch_output_np, onnx_output, rtol=1e-03, atol=1e-05)
print("✅ Las salidas de PyTorch y ONNX son similares.")


c:\Users\Luca\miniconda3\envs\testenv\lib\site-packages\lightning\pytorch\utilities\migration\utils.py:56: The loaded checkpoint was produced with Lightning v2.3.3, which is newer than your current Lightning version: v2.2.5
c:\Users\Luca\miniconda3\envs\testenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Luca\miniconda3\envs\testenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ Las salidas de PyTorch y ONNX son similares.
